# OpenStreetMap Data Case Study

## 1.Map Area

brooklyn, New York, United States

https://mapzen.com/data/metro-extracts/metro/brooklyn_new-york/

The data is extracted from the website above. This is where I am currently live. I am interested to see what what database can reveal.

## 2.Parse Data

### 2.1 Problems encountered in your map 

#### 2.1.1 Phone number

There are many different forms of phone number, such as:

+1 (718) 250-0200; 
+1-718-875-0042; 
+1 718 789-7050; 
1 718 363 7828; 
718 363 7828;

I change them into the form like: +1 7183637828

In [1]:
import re
def update_telephone_form(telephone):
    # The regex matches telephone numbers in this forma:+#-(###)-###-####,###-###-####,+1-###-###-####...
    telephone_patten=re.compile(r'(.*)+(\d\d\d)+(.*)+(\d\d\d)+(.*)+(\d\d\d\d)')
    # The following statement modifies the telephone number as +1 ##########
    return telephone_patten.sub(lambda m: '+1 '+m.group(2)+m.group(4)+m.group(6),telephone)

#### 2.1.2 Street Name

Some street use abbreviation, like: St

Change the abbreviations to the normal words, like Street

St->Street

In [2]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Rode",
            "Ave":"Avenue",
            "Rd.": "Road"
            }
#Change the abbreviations words: st->Street
def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name=re.sub(street_type_re,mapping[street_type],name)
    return name

### 2.2 Overview of data 

#### 2.2.1 Count tags

The basic elements of OpenStreetMap's data model are consist of:

nodes (defining points in space)

ways (defining linear features and area boundaries)

relations (which are sometimes used to explain how other elements work together)

All of the above can have one or more associated tags (which describe the meaning of a particular element)
                                                                                                       

In [3]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET

In [3]:
def count_tags(filename):
    tags={}
    context=ET.iterparse(filename)
    context=iter(context)
    for event,element in context:
        if element.tag not in tags:
            tags[element.tag]=1
        else:
            tags[element.tag]+=1
    return tags
print count_tags('brooklyn_new-york.osm')

{'node': 2485165, 'nd': 3495604, 'bounds': 1, 'member': 14569, 'tag': 2820491, 'relation': 1704, 'way': 490438, 'osm': 1}


#### 2.2.2 Parse data to csv and sql

Parse data to csv and sql table, the csv files and sql tables are listed below:

   element       file             table
    
    node       node.csv           node
    
  node tags    node_tags.csv    node_tags
  
    way         way.csv           way
    
  way node     way_node.csv      way_node
  
  way tags    way_tag.csv       way_tags
  
  
The size of files:

             brooklyn_new-york.osm       666.1Mb

             node.csv                    232 Mb

             node_tags.csv               8.3 Mb

             way.csv                     33 Mb

             way_node.csv                83.4 Mb

             way_tag.csv                 82.6 Mb
             
             map                         405.6 Mb

For details about the code of converting osm to csv and sql files, please see to_csv_and_sql.html

#### 2.2.2 Overview of the data

Number of unique users in node is 1197

Number of unique users in way is 956

Number of total unique users is 1398.

In [2]:
# Number of unique users in node
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('Select count(user) from (select user,count(user) from node group by user)')
results= c.fetchall()
print results

[(1197,)]


In [4]:
# Number of unique users in way
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('Select count(user) from (select user,count(user) from way group by user)')
results= c.fetchall()
print results

[(856,)]


In [10]:
# Number of total unique user
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('''Select count(uid) from 
(select uid from node group by uid 
UNION
select uid from way group by uid)''')
results= c.fetchall()
print results

[(1398,)]


Number of nodes is: 2485165

Number of ways is: 490438

In [11]:
# Number of nodes 
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('Select count(id) from (select id,count(id) from node group by id)')
results= c.fetchall()
print results

[(2485165,)]


In [10]:
# Number of ways 
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('Select count(id) from (select id,count(id) from way group by id)')
results= c.fetchall()
print results

[(490438,)]


Number of types of nodes: 36

The types are like: addr,drink,crossing...

In [12]:
#Number of types of nodes
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('''Select count(type) from 
(select type from node_tag group by type)''')
results= c.fetchall()
print results

[(36,)]


In [13]:
# Example types 
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('''select type from node_tag group by type limit 10''')
results= c.fetchall()
print results

[(u'abandoned',), (u'addr',), (u'building',), (u'contact',), (u'crossing',), (u'description',), (u'diet',), (u'disused',), (u'drink',), (u'exit_to',)]


Top ten attributing users:

In [4]:
# Top ten attributing users
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('''select user,count(id) from 
          (select uid,id,user from node group by id UNION select uid,id,user from way group by id) e
          group by e.uid order by count(id) desc limit 10''')
results= c.fetchall()
print results

[(u'Rub21_nycbuildings', 1740371), (u'ingalls_nycbuildings', 373554), (u'ediyes_nycbuildings', 189694), (u'celosia_nycbuildings', 117361), (u'ingalls', 105358), (u'lxbarth_nycbuildings', 79851), (u'aaron_nycbuildings', 42023), (u'ewedistrict_nycbuildings', 35002), (u'smlevine', 25054), (u'robgeb', 23684)]


In [5]:
import pandas as pd
re=pd.DataFrame(results,columns=['user name','nodes attributed'])
print re

                  user name  nodes attributed
0        Rub21_nycbuildings           1740371
1      ingalls_nycbuildings            373554
2       ediyes_nycbuildings            189694
3      celosia_nycbuildings            117361
4                   ingalls            105358
5      lxbarth_nycbuildings             79851
6        aaron_nycbuildings             42023
7  ewedistrict_nycbuildings             35002
8                  smlevine             25054
9                    robgeb             23684


395 users are appearing only once.

In [26]:
# Number of users appearing only once
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('''select count(usera) from (select user as usera from 
          (select uid,id,user from node group by id UNION All select uid,id,user from way group by id) e
          group by e.user having count(user)==1 )''')
results= c.fetchall()
print results

[(395,)]


 How many cities in this area?

In [10]:
import sqlite3
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('select value from way_tag where key="city" and type="addr" group by value')
results= c.fetchall()
print results

[(u'Brookklyn',), (u'Brooklyn',), (u'Brooklyn ',), (u'Brooklyn, NY',), (u'Corona',), (u'Elmhurst',), (u'Forest Hills',), (u'Glendale',), (u'Glendale, NY',), (u'Hoboken',), (u'Jamaica',), (u'Jersey City',), (u'M',), (u'Maspeth',), (u'Middle Village',), (u'NEW YORK CITY',), (u'New York',), (u'New York City',), (u'Queens',), (u'Rego Park',), (u'Ridgewood',), (u'Rockaway Park',), (u'Waterbury',), (u'Woodside',), (u'brooklyn',), (u'new york',)]


From here we can find that the same city maybe expressed in different ways like 'Brookklyn','Brooklyn','Brooklyn ''Brooklyn, NY', 'Brooklyn, NY'. It will be better if the expression is same.

Different types of amenities:

In [15]:
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('Select value,count(value) from node_tag where key="amenity" group by value order by count(value) desc')
results= c.fetchall()
print results

[(u'bicycle_parking', 2809), (u'restaurant', 856), (u'place_of_worship', 364), (u'school', 358), (u'cafe', 287), (u'bench', 262), (u'bicycle_rental', 261), (u'bar', 176), (u'fast_food', 174), (u'fire_station', 119), (u'bank', 109), (u'pharmacy', 91), (u'toilets', 72), (u'pub', 67), (u'post_box', 61), (u'drinking_water', 55), (u'fuel', 46), (u'parking', 44), (u'atm', 32), (u'hospital', 31), (u'post_office', 28), (u'grave_yard', 26), (u'ferry_terminal', 23), (u'library', 23), (u'theatre', 23), (u'waste_basket', 21), (u'fountain', 20), (u'police', 18), (u'car_sharing', 15), (u'bbq', 12), (u'cinema', 12), (u'marketplace', 11), (u'nightclub', 11), (u'telephone', 10), (u'university', 10), (u'doctors', 9), (u'ice_cream', 8), (u'taxi', 8), (u'college', 7), (u'community_centre', 7), (u'biergarten', 5), (u'childcare', 5), (u'courthouse', 5), (u'dentist', 5), (u'townhall', 5), (u'arts_centre', 4), (u'bicycle_repair_station', 4), (u'car_rental', 4), (u'recycling', 4), (u'clinic', 3), (u'kindergart

In [19]:
import pandas as pd
re=pd.DataFrame(results,columns=['value','count'])
print re.head(8)

              value  count
0   bicycle_parking   2809
1        restaurant    856
2  place_of_worship    364
3            school    358
4              cafe    287
5             bench    262
6    bicycle_rental    261
7               bar    176


What kind of information can we get from resturant

In [21]:
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('select id from node_tag where value="restaurant" limit 2')
results= c.fetchall()
print results

[(380044344,), (447943157,)]


In [23]:
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('select type,value,key from node_tag where id="380044344"')
results= c.fetchall()
print results

[(u'regular', u"Pedro's", u'name'), (u'regular', u'restaurant', u'amenity'), (u'regular', u'mexican', u'cuisine'), (u'regular', u'http://www.pedrosdumbo.net/', u'website'), (u'regular', u"Pedro's", u'name'), (u'regular', u'restaurant', u'amenity'), (u'regular', u'mexican', u'cuisine'), (u'regular', u'http://www.pedrosdumbo.net/', u'website')]


In [24]:
node_tag = sqlite3.connect("map")
c = node_tag.cursor()
c.execute('select type,value,key from node_tag where id="447943157"')
results= c.fetchall()
print results

[(u'regular', u'Peanut Butter & Co', u'name'), (u'regular', u'restaurant', u'amenity'), (u'regular', u'sandwich', u'cuisine'), (u'regular', u'Peanut Butter & Co', u'name'), (u'regular', u'restaurant', u'amenity'), (u'regular', u'sandwich', u'cuisine')]


From this we can see that the keys sometimes are repeated. And the information of the is not completed.

## 3.Other ideas about the datasets

Problems:

1.The number of unique attributing users is only 1398, which is quite small comparing to the population living in brooklyn.
2.The information(tags) about the node are not enough.
3.The tags are in different forms(for example the telephone number) which makes tags hard to analyze.

Suggestion:

1.Post some advertisement and some rewards may be given to the people who attribute to the map
2.Some third party tools like Google Maps may be used to improve the data.
3.When user update tags, suggest them to put the data in a standard form. For example: when people make a tag of phone number, suggest them to put it into the form like: +1 ##########(don't use +1(###)-###-#### or other form).

Benifit of the suggestion:
1.More people are involved in developing the database so we can have more information.
2.The data are performed in a uniform way, which makes it easy to clean and analyze data.

Problems:
Some economic problems may occur.